In [1]:
import os
import sys
import json
import pandas as pd
from keybert import KeyBERT
from bertopic import BERTopic

In [2]:
cwd = !pwd
sys.path.append(os.path.split(cwd[0])[0])

In [3]:
from fantastic_amazon import amazon_scraper, sheets_connector

## Connect to Google Sheets 

In [4]:
SERVICE_FILE = "/home/max/fantastic-amazon-c1ba8f85a23b.json"
SHEET_KEY = "1M8FT1qtKr5VgUAk-7IvQdX8Hp2Sn13uZLL3tq2o7YOU"
SHEET_NAME = "PRICES"

In [5]:
gs = sheets_connector.GoogleSheets(sheet_key=SHEET_KEY, sheet_name=SHEET_NAME, service_file=SERVICE_FILE) 

In [6]:
PNs = gs.w.range("B3:B30")
url_dict = {} 
for PN in PNs:
    url_row = 'H' + str(PN[0].row) + ':J' + str(PN[0].row) 
    urls = gs.get_urls(url_row)
    url_dict[PN[0].value] = urls

In [7]:
url_dict['C31DUAL4KDP']

['https://www.amazon.com/StarTech-com-2-Port-USB-C-DisplayPort-MST/dp/B01KIMP28Y/ref=sr_1_5?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-5',
 'https://www.amazon.com/2-Port-DisplayPort-Adapter-Monitor-Splitter/dp/B07563JV7H/ref=sr_1_8?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-8',
 'https://www.amazon.com/gofanco-DisplayPort-Displayport-Thunderbolt-Compatible/dp/B07MF4QXFX/ref=sr_1_3?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-3']

## Read in the Product Page

In [8]:
with open(os.path.join('..', 'data', 'product_info.json'), 'r') as f:
    prod = json.load(f)

In [9]:
# prod['C31DUAL4KDP']['https://www.amazon.com/StarTech-com-2-Port-USB-C-DisplayPort-MST/dp/B01KIMP28Y/ref=sr_1_5?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-5']'https://www.amazon.com/StarTech-com-2-Port-USB-C-DisplayPort-MST/dp/B01KIMP28Y/ref=sr_1_5?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-5'
PN = 'C31DUAL4KDP'
titles = [prod[PN][l]['title'] for l in url_dict[PN]]
[print (title) for title in titles]

StarTech.com 2-Port Multi Monitor Adapter - USB-C to 2x DisplayPort 1.2 Video Splitter - USB Type-C to DP MST Hub - Dual 4K 30Hz or 1080p 60Hz - Thunderbolt 3 Compatible - Windows Only (MSTCDP122DP)
UPTab USB C to Dual DisplayPort Adapter 8K, UPTab USB-C to DisplayPort Multi Monitor Splitter Converter, (Compatible Thunderbolt 3), for Windows PC MST, macOS Mirror SST only
gofanco USB-C to Dual 4K DisplayPort MST Adapter Splitter, 4K @30Hz, USB Type C to Displayport Multi Monitor Converter, Thunderbolt 3 Compatible, for Windows PCs, NOT Mac, (DP Alternate Mode Required)


[None, None, None]

In [10]:
url_dict[PN]

['https://www.amazon.com/StarTech-com-2-Port-USB-C-DisplayPort-MST/dp/B01KIMP28Y/ref=sr_1_5?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-5',
 'https://www.amazon.com/2-Port-DisplayPort-Adapter-Monitor-Splitter/dp/B07563JV7H/ref=sr_1_8?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-8',
 'https://www.amazon.com/gofanco-DisplayPort-Displayport-Thunderbolt-Compatible/dp/B07MF4QXFX/ref=sr_1_3?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-3']

## Pull First Page of Search Terms

In [11]:
one_search = amazon_scraper.AmazonScrapper().scrape_search("amazon.com", search_terms="hdmi usb multi monitor cable", headless=True)
other_search = amazon_scraper.AmazonScrapper().scrape_search("amazon.com", search_terms="USBC to 4K DisplayPort", headless=True)

In [12]:
df = pd.DataFrame.from_records([value for key, value in one_search.items() if key != 'suggestions' if value['price'] > 25], columns=list(first_page['2'].keys()))
df.head()
o_df = pd.DataFrame.from_records([value for key, value in other_search.items() if key != 'suggestions' if value['price'] > 25], columns=list(first_page['2'].keys()))
o_df.head()

NameError: name 'first_page' is not defined

In [ ]:
doc = "\n".join(df['title'])
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 5), stop_words='english')

In [ ]:
keywords